### immune_submodels.cpp

#### From lines 619-628: Transformation of macrophages from M1 to M2 phenotype

```c++
// If it is not me, not dead and is a CD8+ T cell that is within a very short distance from me, I will stop secreting pro-inflammatory cytokine, transform to M2 phenotype, and start secreting TGF-β
		if( pContactCell != pCell && pContactCell->phenotype.death.dead == false && pContactCell->type == CD8_Tcell_type 
			&& pCell->custom_data["activated_immune_cell"] > 0.5 && cell_cell_distance<=parameters.doubles("epsilon_distance")*(radius_mac+radius_test_cell)) 
		{
			pCell->custom_data["M2_phase"] = 1;
			phenotype.secretion.secretion_rates[proinflammatory_cytokine_index] = 0;// Contact with CD8+ T cell turns off pro-inflammatory cytokine secretion
			pCell->phenotype.secretion.net_export_rates[antiinflammatory_cytokine_index] = pCell->custom_data["antiinflammatory_cytokine_secretion_rate_by_macrophage"];// and turns on anti-inflammatory cytokine secretion
			n=neighbors.size();
		}
```

#### From lines 1095-1132: Function of fibroblasts

```c++
void fibroblast_phenotype( Cell* pCell, Phenotype& phenotype, double dt )
{
	
	static int antiinflammatory_cytokine_index = microenvironment.find_density_index("anti-inflammatory cytokine");
	static int collagen_index = microenvironment.find_density_index("collagen");
	double TGF_beta = (pCell->nearest_density_vector()[antiinflammatory_cytokine_index])*1e12;
	static int apoptosis_index = phenotype.death.find_death_model_index( "Apoptosis" );
	static Cell_Definition* pCD = find_cell_definition( "fibroblast" );

	// no apoptosis until activation for homeostasis
	if( pCell->custom_data["activated_immune_cell"] < 0.5 )
	{ phenotype.death.rates[apoptosis_index] = 0.0; }
	else
	{ phenotype.death.rates[apoptosis_index] = pCD->phenotype.death.rates[apoptosis_index]; }			
	
	if( phenotype.death.dead == true )
	{
		pCell->functions.update_phenotype = NULL;
		pCell->functions.custom_cell_rule = NULL;
		return; 
	}
	pCell->phenotype.secretion.net_export_rates[collagen_index] = (((0.942*TGF_beta)/(0.174+TGF_beta))*(pCell->custom_data["collagen_secretion_rate"])*2.52e-7);


    for( int n=0; n<microenvironment.mesh.voxels.size(); n++ )
    {
        double TGF_beta = (microenvironment(n)[antiinflammatory_cytokine_index])*1e12;

        if( TGF_beta > 0 )
        {
            pCell->custom_data["activated_immune_cell"] = 1.0;

            return;
         }
    }
	
	return;
}
```

#### From lines 1154-1161: Function of secreting agents created at the damaged sites

```c++
void residual_phenotype( Cell* pCell, Phenotype& phenotype, double dt )
{

    static int antiinflammatory_cytokine_index = microenvironment.find_density_index("anti-inflammatory cytokine");
    //pCell->phenotype.secretion.secretion_rates[antiinflammatory_cytokine_index] = pCell->custom_data["antiinflammatory_cytokine_secretion_rate_by_damagedSite"];
    pCell->phenotype.secretion.net_export_rates[antiinflammatory_cytokine_index] = pCell->custom_data["antiinflammatory_cytokine_secretion_rate_by_damagedSite"];
	return;
}
```

#### From lines 1658-1709: Function for the recruitment of fibroblasts

```c++
//  fibroblast recruitment
		static double fibroblast_recruitment_rate = parameters.doubles( "fibroblast_max_recruitment_rate" );
		static double f_min_signal = parameters.doubles( "fibroblast_recruitment_min_signal" );
		static double f_sat_signal = parameters.doubles( "fibroblast_recruitment_saturation_signal" );
		static double f_max_minus_min = f_sat_signal - f_min_signal;

		total_rate = 0;
		// integrate \int_domain r_max * (signal-signal_min)/(signal_max-signal_min) * dV
		total_scaled_signal= 0.0;
		for( int n=0; n<microenvironment.mesh.voxels.size(); n++ )
		{
			// (signal(x)-signal_min)/(signal_max/signal_min)
			double TGF_beta = (microenvironment(n)[antiinflammatory_cytokine_index])*1e12;
			double dRate = ( 0.0492*pow(TGF_beta,3) -0.9868*pow(TGF_beta,2) +6.5408*TGF_beta + 7.1092 - f_min_signal );
			dRate /= f_max_minus_min;
			// crop to [0,1]
			if( dRate > 1 )
			{ dRate = 1; }
			if( dRate < 0 )
			{ dRate = 0; }
			total_rate += dRate;
		}
		
		// multiply by dV and rate_max
		total_scaled_signal = total_rate;

		total_rate *= microenvironment.mesh.dV;
		total_rate *= fibroblast_recruitment_rate;
		
		// expected number of new fibroblast
		number_of_new_cells_prob = total_rate * elapsed_time;
		number_of_new_cells_int = floor( number_of_new_cells_prob );
	    alpha = number_of_new_cells_prob - number_of_new_cells_int;	
		
		//STOCHASTIC PORTION		
	    
	    if( UniformRandom()< alpha )
	    {
			number_of_new_cells_int++;
	    }
		recruited_fibroblasts += number_of_new_cells_int;
		
		if( number_of_new_cells_int )
		{
			if( t_immune < first_fibroblast_cell_recruitment_time )
			{ first_fibroblast_cell_recruitment_time = t_immune; }

			//std::cout << "\tRecruiting " << number_of_new_cells_int << " fibroblast cells ... " << std::endl;

			for( int n = 0; n < number_of_new_cells_int ; n++ )
			{ create_infiltrating_fibroblast(); }
		}
```